In [1]:
from typing import List
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from toy_text_adventure import find_the_keys
from simplest_q_learner import q_learner

ModuleNotFoundError: No module named 'typeing'

## test the game

In [ ]:
game = find_the_keys()

In [ ]:
game._get_state()

In [ ]:
game.get_description()

In [ ]:
game.get_reward()

In [ ]:
game.get_actions()

In [ ]:
game.do('go left')

In [ ]:
game.get_actions()

In [ ]:
game.do('go back')

In [ ]:
game.get_actions()

In [ ]:
# how to solve
game.do('go straight')
game.do('enter house')
game.do('check cupboard')
print(game.reward)

# random exploration

In [ ]:
game = find_the_keys()
steps = 900
rewards_random = []
rng = np.random.default_rng()
for i in range(steps):
    description = game.get_description()
    actions = game.get_actions()
    rewards_random.append(game.get_reward())
    #print(description, reward)
    if rewards_random[-1]:
        # this only makes sense because of the specific game here
        #print("You won! Reseting game.")
        game.reset()
    else:
        j = rng.integers(0, len(actions))
        #print("do", actions[j][0])
        game.do(actions[j])

In [ ]:
sum(rewards_random)

## q-learning

In [ ]:
# again the most naive way I could think of, see https://en.wikipedia.org/wiki/Q-learning
agent = q_learner()
game = find_the_keys()

state = game.get_description()
actions = game.get_actions()

steps = 900
rewards_q = []
verbose=False
agent.verbose=verbose

for i in range(steps):
    if verbose: print()
    if verbose: print("step", i, "state", state)
    
    action, action_q = agent.choose_action(state, actions)
    
    if verbose: print("do", action)
    game.do(action)
    
    new_state = game.get_description()
    new_reward = game.get_reward()
    new_actions = game.get_actions()
    
    if verbose: print("new reward", new_reward, "max new q", max_new_q)
    
    agent.learn(state, action, action_q, new_state, new_reward, new_actions)
    if verbose: print("q", agent.q)
    
    rewards_q.append(new_reward)
    
    # this particular game should be reset after the reward is found
    if new_reward:
        game.reset()
        new_state = game.get_description()
        new_actions = game.get_actions()
    
    # prepare for next iteration
    state = new_state
    actions = new_actions  

In [ ]:
sum(rewards_q)

In [ ]:
fig1 = plt.figure(num=1)
ax1 = fig1.gca()
ax1.plot(np.cumsum(rewards_random), label='random exploration')
ax1.plot(np.cumsum(rewards_q), label='q learner')
ax1.set_ylabel('total reward')
ax1.set_xlabel('steps')
ax1.legend()